In [1]:
import filehandling
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def corr(features, boundary, r_min, r_max, dr):
    radius = features.r.mean()
    area = calculate_area_from_boundary(boundary)
    N = features.x.count()
    density = N / area

    r_values = np.arange(r_min, r_max, dr) * radius

    dists, orders, Nt, N = dists_and_orders(features, r_max * radius)
    g, bins = np.histogram(dists, bins=r_values)
    g6, bins = np.histogram(dists, bins=r_values, weights=orders)

    bin_centres = bins[1:] - (bins[1] - bins[0]) / 2

    dr = bins[1]-bins[0]
    annulus_areas = 2 * np.pi * r_values[:-1] * dr
    expected_number = annulus_areas * density
    factor = 1 / (expected_number * (Nt-1))

    g = g *factor
    g6 = g6 * factor
    return bin_centres, g, g6

In [3]:
def dists_and_orders(f, t=1000):
    idx = get_idx(f, t)
    dists = get_dists(f, idx)
    orders = get_orders(f, idx)
    return dists.ravel(), orders.ravel(), dists.shape[0], dists.shape[1]


def get_idx(f, t):
    return f.edge_distance.values > t


def get_dists(f, idx):
    x = f[['x', 'y']].values
    return spatial.distance.cdist(x[idx, :], x)


def get_orders(f, idx):
    orders = make_complex(f)
    order_grid = make_order_grid(orders, idx)
    return np.abs(order_grid)


def make_order_grid(orders, idx):
    return orders[idx] @ np.conj(orders).transpose()


def make_complex(f):
    return f[['order_r_long']].values + 1j * f[['order_i_long']].values


def flat_array(x):
    return np.concatenate([item.ravel() for item in x])

In [4]:
def calculate_area_from_boundary(boundary):

    x, y = sort_polygon_vertices(boundary)
    area = calculate_polygon_area(x, y)
    return area

def calculate_polygon_area(x, y):
    p1 = 0
    p2 = 0
    for i in range(len(x)):
        p1 += x[i] * y[i-1]
        p2 += y[i] * x[i-1]
    area = 0.5 * abs(p1-p2)
    return area


def sort_polygon_vertices(points):
    cx = np.mean(points[:, 0])
    cy = np.mean(points[:, 1])
    angles = np.arctan2((points[:, 1] - cy), (points[:, 0] - cx))
    sort_indices = np.argsort(angles)
    x = points[sort_indices, 0]
    y = points[sort_indices, 1]
    return x, y

def get_boundary(data):
    hull = spatial.ConvexHull(data.loc[0, ['x', 'y']].values)
    boundary = hull.points[hull.vertices]
    return boundary

In [8]:
files = filehandling.get_directory_filenames("/media/data/Data/N29/FirstOrder/Susceptibility/Dimpled2/*.hdf5")

In [9]:
for i, file in tqdm(enumerate(files)):
    data = pd.read_hdf(file)
    if i == 0:
        boundary = get_boundary(data)
    r, g, g6 = corr(data.loc[0], boundary, 1, 20, 0.2)
    duty = round(data.Duty.mean())
    np.savetxt(f"/media/data/Data/N29/FirstOrder/Correlations/November2022/Dimpled/{duty}_r.txt", r)
    np.savetxt(f"/media/data/Data/N29/FirstOrder/Correlations/November2022/Dimpled/{duty}_g.txt", g)
    np.savetxt(f"/media/data/Data/N29/FirstOrder/Correlations/November2022/Dimpled/{duty}_g6.txt", g6)

100it [02:26,  1.46s/it]


In [5]:
files_flat = filehandling.get_directory_filenames("/media/data/Data/N29/FirstOrder/Susceptibility/Flat2/*.hdf5")

In [6]:
for i, file in tqdm(enumerate(files_flat)):
    data = pd.read_hdf(file)
    if i == 0:
        boundary = get_boundary(data)
    r, g, g6 = corr(data.loc[0], boundary, 1, 20, 0.2)
    duty = round(data.Duty.mean())
    np.savetxt(f"/media/data/Data/N29/FirstOrder/Correlations/November2022/Flat/{duty}_r.txt", r)
    np.savetxt(f"/media/data/Data/N29/FirstOrder/Correlations/November2022/Flat/{duty}_g.txt", g)
    np.savetxt(f"/media/data/Data/N29/FirstOrder/Correlations/November2022/Flat/{duty}_g6.txt", g6)

100it [03:48,  2.28s/it]
